In [ ]:
import fastf1 as ff1
import pandas as pd
import numpy as np
import datetime
from fastf1 import plotting
from matplotlib import pyplot as plt

plotting.setup_mpl()

ff1.Cache.enable_cache('cache')  # optional but recommended

In [ ]:
class F1_graphs:
    
    season = 0
    session = None
    laps = None
    gp_name = None
    session_name = None
    
    def __init__(self, season, grandprix, session) :
        self.season = season
        self.session = ff1.get_session(season, grandprix, session)
        self.session.load()
        self.laps = self.session.laps
        self.gp_name = self.session.event['EventName']
        self.session_name = self.session.name
    
    def get_laps_for_driver(self, driver):
        return self.laps.pick_driver(driver)
    
    def get_laps_for_drivers(self, drivers):
        combined = pd.DataFrame()
        for driver in drivers:
            laps = helper.get_laps_for_driver(driver)
            laps = laps[['Driver', 'LapNumber', 'Stint', 'LapTime']]
            combined = combined.append(laps).reset_index(drop=True)
        return combined
    
    def plot_lap_times(self, drivers, filter=False, ymin=None, ymax=None):
        laptimes = helper.get_laps_for_drivers(drivers)
        fig, ax = plt.subplots(figsize=(15,10))
        ax.set_xlabel('Lap')
        ax.set_ylabel('Lap time')
        plt.title(f"Lap times")

        for driver in drivers:
            color = plotting.driver_color(driver)
            laps = laptimes[laptimes.Driver == driver]
            if filter:
                limit = laps.LapTime.mean() * 1.01
                laps['LapTime'].values[laps['LapTime'].values > limit] = None
            ls = '-'
            ax.plot(laps.LapNumber, laps.LapTime, color=color, label=driver, ls=ls)
        ax.legend()
        if ymin:
            ymin=pd.Series([ymin])
            ymi=pd.to_timedelta((ymin.str.split(':', expand=True).astype(int) * (60, 1)).sum(axis=1), unit='sec')[0]
            ymax=pd.Series([ymax])
            yma=pd.to_timedelta((ymax.str.split(':', expand=True).astype(int) * (60, 1)).sum(axis=1), unit='sec')[0]
            ax.set_ylim(ymi, yma)
        plt.show()

    def get_section_time(self, tellap, start, end):
        if start:
            tellap = tellap[(tellap.Distance >= start) & (tellap.Distance <= end)]
        return (tellap.Time.max() - tellap.Time.min()).total_seconds()
        
    def compare_laps(self, driver_laps, startpoint=None, endpoint=None, title=None):
        fig, ax = plt.subplots(4, sharex=True, figsize=(20,15))
        ax[0].set_ylabel('Speed', fontsize=16)
        ax[1].set_ylabel('Throttle', fontsize=16)
        ax[2].set_ylabel('Brake', fontsize=16)
        ax[3].set_ylabel('Gear', fontsize=16)
        ax[3].set_xlabel("meter")

        if startpoint:
            ax[1].set_xlim(startpoint, endpoint)
        lw = 4 if startpoint else 1
        
        drivers_plotted = []
        timings = []
        for dl in driver_laps:
            driver = dl['driver']
            lap = dl['lap']
            lap = self.laps.pick_driver(driver).pick_fastest().LapNumber if 'fastest' == lap else lap
            tellap = self.laps[(self.laps.LapNumber == lap) & (self.laps.Driver == driver)].get_telemetry()
            timing = self.get_section_time(tellap, startpoint, endpoint)
            timings.append(timing)
            color = plotting.driver_color(driver)
            label = driver + ' (' + str(lap) + ')'
            label = "{} (lap {}, {:5.2f}s)".format(driver, lap, timing)
            ls='-' if driver not in drivers_plotted else ':'
            ax[0].plot(tellap['Distance'], tellap['Speed'], color=color, label=label, lw=lw, ls=ls)
            ax[1].plot(tellap['Distance'], tellap['Throttle'], color=color, label=label, lw=lw, ls=ls)
            ax[2].plot(tellap['Distance'], tellap['Brake'], color=color, label=label, lw=lw, ls=ls)
            ax[3].plot(tellap['Distance'], tellap['nGear'], color=color, label=label, lw=lw, ls=ls)
            drivers_plotted.append(driver)

        if not title:
            title = 'Comparison'
            if len(timings) == 2:
                title = 'Comparison, diff {} to {} = {:5.3f}s'.format(driver_laps[0]['driver'], 
                                                                      driver_laps[1]['driver'], 
                                                                      timings[0] - timings[1])
        ax[0]. set_title(title)
        ax[0].legend()
        plt.show()    

    def compare_laps_auto_color(self, driver_laps, startpoint=None, endpoint=None, title=None):
        fig, ax = plt.subplots(4, sharex=True, figsize=(20,15))
        ax[0].set_ylabel('Speed', fontsize=16)
        ax[1].set_ylabel('Throttle', fontsize=16)
        ax[2].set_ylabel('Brake', fontsize=16)
        ax[3].set_ylabel('Gear', fontsize=16)
        ax[3].set_xlabel("meter")

        if startpoint:
            ax[1].set_xlim(startpoint, endpoint)
        lw = 4 if startpoint else 1
        
        drivers_plotted = []
        timings = []
        for dl in driver_laps:
            driver = dl['driver']
            lap = dl['lap']
            lap = self.laps.pick_driver(driver).pick_fastest().LapNumber if 'fastest' == lap else lap
            tellap = self.laps[(self.laps.LapNumber == lap) & (self.laps.Driver == driver)].get_telemetry()
            timing = self.get_section_time(tellap, startpoint, endpoint)
            timings.append(timing)
            label = driver + ' (' + str(lap) + ')'
            label = "{} (lap {}, {:5.2f}s)".format(driver, lap, timing)
            ax[0].plot(tellap['Distance'], tellap['Speed'], label=label, lw=lw)
            ax[1].plot(tellap['Distance'], tellap['Throttle'], label=label, lw=lw)
            ax[2].plot(tellap['Distance'], tellap['Brake'], label=label, lw=lw)
            ax[3].plot(tellap['Distance'], tellap['nGear'], label=label, lw=lw)
            drivers_plotted.append(driver)
            
        if not title:
            title = 'Comparison'
            if len(timings) == 2:
                title = 'Comparison, diff {} to {} = {:5.3f}s'.format(driver_laps[0]['driver'], 
                                                                      driver_laps[1]['driver'], 
                                                                      timings[0] - timings[1])
        ax[0]. set_title(title)
        ax[0].legend()
        plt.show()    

    def plot_circuit(self, min_x=None, max_x=None, min_y=None, max_y=None):
        tellap = self.laps.iloc[0].get_telemetry()
        if min_x:
            tellap = tellap[tellap.X >= min_x]
        if max_x:
            tellap = tellap[tellap.X <= max_x]
        if min_y:
            tellap = tellap[tellap.Y >= min_y]
        if max_y:
            tellap = tellap[tellap.Y <= max_y]
        
        fig, ax = plt.subplots(figsize=(8,8))
        ax.plot(tellap['X'], tellap['Y'], color='white', linestyle='-', linewidth=4)
        
        title = ('{} (from {:6.1f} m to {:6.1f} m)'.format(self.gp_name, tellap['Distance'].min(), 
                                                           tellap['Distance'].max()))
        ax.set_title(title)
        
    def plot_cummulative_diff(self, drivers, ref_driver=None, ref_time=None):
        # Get laps of reference driver or winner
        if not ref_driver:
            ref_driver = self.session.results.iloc[0]['Abbreviation']
            title = "Difference to race winner"
        else:
            title = "Difference to {}".format(ref_driver)
        ref_laps = self.laps[helper.laps.Driver == ref_driver][['Time']][1:].copy()

        if ref_time:
            t = datetime.datetime.strptime(ref_time,"%M:%S")
            delta = datetime.timedelta(minutes=t.minute, seconds=t.second)
            ref_laps['Time'] = delta
            ref_laps.iloc[0]['Time'] = self.laps[self.laps.LapNumber == 1]['Time'].max()
            ref_laps['Time'] = ref_laps['Time'].cumsum()
            title = "Cummulative delta to baseline of {}s".format(ref_time)
        drivers.append(ref_driver)
        
        fig, ax = plt.subplots(figsize=(15,10))
        ax.set_xlabel('Lap')
        ax.set_ylabel('Seconds')
        plt.title(title)

        for driver in drivers: 
                driver_laps = self.laps[helper.laps.Driver == driver][1:].copy()
                driver_laps['RefTime'] = ref_laps['Time'].to_numpy() / np.timedelta64(1, 's')
                driver_laps['Time'] = driver_laps['Time'] / np.timedelta64(1, 's')
                driver_laps['DeltaTime'] = driver_laps['RefTime'] - driver_laps['Time']
                
                color = plotting.driver_color(driver)
                label = driver
                ax.plot(driver_laps['LapNumber'], driver_laps['DeltaTime'], color=color, label=label)
        
        ax.legend()
        plt.show()
        
    def get_fastest_laps(self):
        fastest_laps = self.laps[['LapTime', 'Driver']].groupby('Driver').min().sort_values('LapTime').reset_index()
        fastest_laps['LapTime'] = fastest_laps['LapTime'].dt.total_seconds()
        return fastest_laps
    
    def get_fastest_laps_per_stint(self):
        tmp=self.laps[['Driver', 'Stint', 'LapTime']].copy()
        tmp['LapTime'] = tmp['LapTime'] / np.timedelta64(1, 's')
        tmp=tmp.groupby(['Driver', 'Stint']).aggregate('min').unstack().reset_index()
        tmp.columns = ['Stint ' + str(col[1]).strip() for col in tmp.columns.values]
        tmp.columns.values[0] = 'Driver'
        return tmp
    
    def get_best_possible_time_per_compound(self):
        tmp = self.laps[['Sector1Time', 'Sector2Time', 'Sector3Time', 'Compound']].copy()
        tmp['Sector1Time'] = tmp['Sector1Time'] / np.timedelta64(1, 's')
        tmp['Sector2Time'] = tmp['Sector2Time'] / np.timedelta64(1, 's')
        tmp['Sector3Time'] = tmp['Sector3Time'] / np.timedelta64(1, 's')
        tmp = tmp.groupby('Compound').min().reset_index()
        tmp['Best Possible Time'] = tmp['Sector1Time'] + tmp['Sector2Time'] + tmp['Sector3Time']
        return tmp[['Compound', 'Best Possible Time']].sort_values('Best Possible Time')
    
    def get_best_possible_time_per_driver(self):
        tmp = self.laps[['Sector1Time', 'Sector2Time', 'Sector3Time', 'Driver']].copy()
        tmp['Sector1Time'] = tmp['Sector1Time'] / np.timedelta64(1, 's')
        tmp['Sector2Time'] = tmp['Sector2Time'] / np.timedelta64(1, 's')
        tmp['Sector3Time'] = tmp['Sector3Time'] / np.timedelta64(1, 's')
        tmp = tmp.groupby('Driver').min().reset_index()
        tmp['Best Possible Time'] = tmp['Sector1Time'] + tmp['Sector2Time'] + tmp['Sector3Time']
        return tmp[['Driver', 'Best Possible Time']].sort_values('Best Possible Time').reset_index(drop=True)

    def plot_teldata_on_map(self, driver, item):
        lap = self.laps.pick_driver(driver).pick_fastest()
        
        x = lap.telemetry['X']              # values for x-axis
        y = lap.telemetry['Y']              # values for y-axis
        color = lap.telemetry[item]      # value to base color gradient on
        points = np.array([x, y]).T.reshape(-1, 1, 2)
        segments = np.concatenate([points[:-1], points[1:]], axis=1)
        # We create a plot with title and adjust some setting to make it look good.
        fig, ax = plt.subplots(sharex=True, sharey=True, figsize=(12, 6.75))
        fig.suptitle(f'{driver} - {item}', size=16, y=0.97)
        # Adjust margins and turn of axis
        plt.subplots_adjust(left=0.1, right=0.9, top=0.9, bottom=0.12)
        ax.axis('off')
        # After this, we plot the data itself.
        # Create background track line
        ax.plot(lap.telemetry['X'], lap.telemetry['Y'], color='black', linestyle='-', linewidth=16, zorder=0)
        # Create a continuous norm to map from data points to colors
        norm = plt.Normalize(color.min(), color.max())
        from matplotlib.collections import LineCollection
        colormap = plt.cm.plasma
        lc = LineCollection(segments, cmap=colormap, norm=norm, linestyle='-', linewidth=5)
        # Set the values used for colormapping
        lc.set_array(color)
        # Merge all line segments together
        line = ax.add_collection(lc)
        # Finally, we create a color bar as a legend.
        from matplotlib import colors, colorbar
        cbaxes = fig.add_axes([0.25, 0.05, 0.5, 0.05])
        normlegend = colors.Normalize(vmin=color.min(), vmax=color.max())
        legend = colorbar.ColorbarBase(cbaxes, norm=normlegend, cmap=colormap, orientation="horizontal")
        # Show the plot
        plt.show()        

    def plot_speed_on_map(self, driver):
        return self.plot_teldata_on_map(driver, 'Speed')

    def plot_throttle_on_map(self, driver):
        return self.plot_teldata_on_map(driver, 'Throttle')

    def plot_brake_on_map(self, driver):
        return self.plot_teldata_on_map(driver, 'Brake')

    def plot_gear_on_map(self, driver):
        return self.plot_teldata_on_map(driver, 'nGear')
    
    def plot_fastest_laps_relative(self):
        from timple.timedelta import strftimedelta
        drivers = pd.unique(self.session.laps['Driver'])
        list_fastest_laps = list()
        for drv in drivers:
            drvs_fastest_lap = self.session.laps.pick_driver(drv).pick_fastest()
            drvs_fastest_lap['Color'] = ff1.plotting.team_color(drvs_fastest_lap['Team'])
            list_fastest_laps.append(drvs_fastest_lap)
        fastest_laps = pd.DataFrame(list_fastest_laps).sort_values(by='LapTime').reset_index(drop=True)
        fastest_laps['LapTimeDelta'] = fastest_laps['LapTime'] -  fastest_laps['LapTime'].min()
    
        fig, ax = plt.subplots(figsize=(15,10))
        ax.bar(fastest_laps.index, fastest_laps['LapTimeDelta'], color=fastest_laps['Color'], edgecolor='grey')
        ax.set_xticks(fastest_laps.index)
        ax.set_xticklabels(fastest_laps['Driver'])

        x_list = fastest_laps.index.to_list()
        y_list = fastest_laps['LapTimeDelta'].to_list()
        for i in range(0, len(x_list)):
            plt.text(i,y_list[i],strftimedelta(y_list[i], "%s.%ms"), ha="center")

        ax.set_ylabel('Seconds')
        ax.set_title('Comparing to fastest lap of {} ({})'.format(strftimedelta(fastest_laps.iloc[0]['LapTime'],
                                                                                '%m:%s.%ms'),
                                                                  fastest_laps.iloc[0]['Driver'],))
        plt.show()
        
    def print_tyre_degredation(self, min_count=3, correct_fuel=True):
        # Only use data from laps with a green flag
        tyredev = self.laps[helper.laps.TrackStatus == '1']
        #Keep relevant columns
        tyredev= tyredev[['Compound', 'TyreLife', 'LapNumber', 'LapTime', 'Sector1Time', 'Sector2Time', 'Sector3Time']]

        # Each kg of fuel adds 0.03s to laptime. Fuel is 110kg at start of race and 0 at finish
        # Recalculate all lap times to cars with no fuel
        if correct_fuel:
            nolaps = tyredev.LapNumber.max()
            totfuel = 110
            secperkg = 0.03
            tyredev['Fuel'] = totfuel - (tyredev['LapNumber'] * (totfuel / nolaps))
            tyredev['LapTime'] = tyredev['LapTime'] / np.timedelta64(1, 's') - (secperkg * tyredev['Fuel'])

        # Drop first lap of the race and first track on set of tyres
        tyredev = tyredev[(tyredev.TyreLife > 1) & (tyredev.LapNumber > 1)]

        # Get minimum laptime per [Compound, TyreLife] and count of occurences of this combination
        tyredev = tyredev[['Compound', 'TyreLife', 'LapTime']].groupby(['Compound', 'TyreLife']). \
                  agg({'LapTime': ['min', 'count']}).reset_index()
        tyredev.columns = ['Compound', 'TyreLife', 'LapTime', 'Count']

        # Only use combinations with a mimimum count ans remove first laps on tyre
        tyredev = tyredev[(tyredev['Count'] >= min_count) & (tyredev['TyreLife'] > 1)]

        fig, ax = plt.subplots(figsize=(15,10))
        for tyre,color in zip(['SOFT', 'MEDIUM', 'HARD', 'INTERMEDIATE', 'WET'], ['red', 'yellow', 'white', 'green', 'blue']):
            df = tyredev[(tyredev['Compound'] == tyre)].dropna()
            if not df.empty:
                df.plot('TyreLife', 'LapTime', ax=ax, color=color, label=tyre)
        _ = ax.set_ylim((int(tyredev.LapTime.min()/10))*10, (int(tyredev.LapTime.max()/10)+1)*10)
        _ = ax.set_title('Tyre degradation')
        _ = ax.set_ylabel('Laptime (s)')

    def print_tyre_strategies(self):
        fig, ax = plt.subplots(figsize=(15,10))

        pitstops = self.laps[['LapNumber', 'Stint', 'Driver']].copy()
        pitstops['Pitstop'] = ~pitstops.Stint.eq(pitstops.Stint.shift())
        pitstops = pitstops[pitstops.Pitstop] 
        pitstops = pitstops[pitstops.LapNumber > 1]
        pitstops.plot.scatter('LapNumber', 'Driver', ax=ax, color='purple', s=128)

        df = self.laps[['Driver', 'LapNumber', 'Compound']]
        df = pd.merge(self.session.results[['Abbreviation']], df, left_on='Abbreviation', right_on='Driver')
        for tyre,color in zip(['SOFT', 'MEDIUM', 'HARD', 'INTERMEDIATE', 'WET'], ['red', 'yellow', 'white', 'green', 'blue']):
            df = self.laps[self.laps.Compound == tyre][['Driver', 'LapNumber', 'Compound']]
            df.plot.scatter('LapNumber', 'Driver', ax=ax, color=color, s=16)
        ax.invert_yaxis()
        ax.set_title('Compound usage')
        plt.show()

    def print_pitstop_time_in_pits(self):
        tmp = self.laps[['Driver', 'LapNumber', 'Stint', 'PitOutTime','PitInTime', 'Team']].copy()
        tmp['TimeLost'] = (tmp['PitOutTime'] - tmp['PitInTime'].shift()) / np.timedelta64(1, 's')
        tmp = tmp[['Driver', 'Team', 'LapNumber', 'Stint', 'TimeLost']].dropna()
        color_list = []
        for _, row in tmp.iterrows():
            color_list.append(ff1.plotting.team_color(row['Team']))
        tmp['Color'] = color_list

        fig, ax = plt.subplots(figsize=(15,10))
        ax.scatter(tmp['Driver'].to_list(), tmp['TimeLost'].to_list(), 64, tmp['Color'].to_list())
        ax.axhline(tmp['TimeLost'].mean())
        ax.set_title('Time in pits during pitstop')
        ax.set_ylabel('Seconds')
        plt.show()
        
    def print_pitstop_time_per_team(self):
        tmp = self.laps[['Driver', 'LapNumber', 'Stint', 'PitOutTime','PitInTime', 'Team']].copy()
        tmp['TimeLost'] = (tmp['PitOutTime'] - tmp['PitInTime'].shift()) / np.timedelta64(1, 's')
        tmp = tmp[['Driver', 'Team', 'LapNumber', 'Stint', 'TimeLost']].dropna()
        color_list = []
        for _, row in tmp.iterrows():
            color_list.append(ff1.plotting.team_color(row['Team']))
        tmp['Color'] = color_list
        avg = tmp['TimeLost'].mean()
        tmp = tmp[['Team', 'Color', 'TimeLost']].groupby(['Team', 'Color']).mean().reset_index().sort_values('TimeLost')

        fig, ax = plt.subplots(figsize=(15,10))
        ax.bar(tmp['Team'].to_list(), tmp['TimeLost'].to_list(), color=tmp['Color'].to_list())
        ax.axhline(avg)
        ax.set_title('Average Time in pits during pitstop')
        ax.set_ylabel('Seconds')
        ax.set_ylim(15,30)
        plt.show()   

    def plot_top_speed(self):
        drslist = []
        for driver in self.laps.Driver.unique():
            drs = self.laps.pick_driver(driver).get_telemetry()[['Speed', 'DRS']].groupby('DRS').max()
            withoutDRS = drs[drs.index < 5]['Speed'].max()
            withDRS = drs[drs.index > 5]['Speed'].max()
            drslist.append({'Driver':driver, 'DRS':withDRS, 'noDRS': withoutDRS})
        topspeeds = pd.DataFrame(drslist)

        fig, ax = plt.subplots(figsize=(15,10))
        topspeeds.plot.scatter('Driver', 'DRS', ax=ax, color='red', s=16, label='DRS')
        topspeeds.plot.scatter('Driver', 'noDRS', ax=ax, color='yellow', s=16, label='no DRS')
        ax.axhline(topspeeds.DRS.mean(), color='red', lw=0.5)
        ax.axhline(topspeeds.noDRS.mean(), color='yellow', lw=0.5)
        ax.set_title('Top Speed')
        ax.legend()
        plt.show()

    def plot_distance_between(self, d1, d2):
        tellist = []
        for driver in [d1, d2]:
            driverlaplist = []
            for lap in range(1, self.laps.LapNumber.max()+1):
                lapdata = self.laps[(self.laps.LapNumber == lap) & (self.laps.Driver == driver)]
                tellap = lapdata.get_telemetry()
                tellap['Driver'] = driver
                tellap['LapNumber'] = lap
                tellap = pd.merge(tellap, lapdata, on=['Driver', 'LapNumber'])
                driverlaplist.append(tellap)
            driverlaps = pd.concat(driverlaplist)
            driverlaps.SessionTime =  driverlaps.SessionTime - driverlaps.LapStartTime.min()
            driverlaps['TotalDistance'] = ((driverlaps['LapNumber'] - 1) * driverlaps['Distance'].max()) + driverlaps['Distance']
            tellist.append(driverlaps)
        teldata = pd.concat(tellist)

        df = \
        pd.merge(
            teldata[teldata.Driver == d1][['Driver', 'SessionTime', 'TotalDistance']].rename(columns={'TotalDistance': 'D1_dist'}),
            teldata[teldata.Driver == d2][['Driver', 'SessionTime', 'TotalDistance']].rename(columns={'TotalDistance': 'D2_dist'}),
            how='outer'
        ).sort_values('SessionTime').drop('Driver', axis=1).set_index('SessionTime').interpolate('time').resample('60s').mean()
        df['Distance'] = df['D1_dist'] - df['D2_dist']
        df.index = df.index / np.timedelta64(1, 's')

        fig, ax = plt.subplots(figsize=(15,10))
        df['Distance'].plot()
        ax.axhline(0)
        ax.set_title('Distance on track between {} and {} (+ = ahead)'.format(d1, d2))
        ax.set_ylabel('Distane (meter)')
        ax.set_xlabel('Session Time (s)')

    def get_pitstop_info(self):
        pitstops = self.laps[['LapNumber', 'Stint', 'Team', 'Driver', 'LapTime', 'Compound', 'PitOutTime']].copy()
        pitstops= pitstops.rename(columns={'Compound': 'CompoundNew', 'LapTime': 'OutLap'})
        pitstops[['CompoundOld', 'InLap', 'PitInTime']] = self.laps[['Compound', 'LapTime', 'PitInTime']].shift()
        pitstops['Pitstop'] = ~pitstops.Stint.eq(pitstops.Stint.shift())
        pitstops['StopCount'] = pitstops['Stint'] - 1
        pitstops = pitstops[(pitstops.Pitstop) & (pitstops.LapNumber > 1)]
        pitstops['InLap'] /= np.timedelta64(1, 's')
        pitstops['OutLap'] /= np.timedelta64(1, 's')
        pitstops['TwoLapTime'] = pitstops['InLap'] + pitstops['OutLap']
        pitstops['TimeInPit'] = (pitstops['PitOutTime'] - pitstops['PitInTime']) / np.timedelta64(1, 's')
        pitstops = pitstops.sort_values(['Driver', 'StopCount'])
        pitstops = pitstops[['Driver', 'Team', 'StopCount', 'LapNumber', 'InLap', 'OutLap', 'TimeInPit', 'TwoLapTime',
                             'CompoundOld', 'CompoundNew', 'PitInTime', 'PitOutTime']]
        return pitstops

    def get_pitstop_timelost(self, drivers, nstops=2):
        pitstops = self.get_pitstop_info()
        pitstops = pitstops[pitstops.StopCount <= nstops][['Driver', 'Team', 'TwoLapTime']].groupby(['Driver', 'Team']).sum().reset_index()
        color_list = []
        for _, row in pitstops.iterrows():
            color_list.append(ff1.plotting.team_color(row['Team']))
        pitstops['Color'] = color_list
        pitstops = pitstops.sort_values('TwoLapTime').reset_index()
        pitstops = pitstops[pitstops.Driver.isin(drivers)]
        pitstops['TimeLost'] = pitstops['TwoLapTime'] - pitstops['TwoLapTime'].min()
        return pitstops[['Driver', 'TimeLost', 'Color']]

    def plot_pitstop_timelost(self, drivers, nstops=2):
        df=self.get_pitstop_timelost(drivers, nstops)
        fig, ax = plt.subplots(figsize=(15,10))
        df.plot.bar('Driver', 'TimeLost', color=df['Color'], ax=ax)
        ax.set_ylabel('seconds')
        ax.set_title('Time lost during {} pitstops (in- and outlaps)'.format(2))
        ax.get_legend().remove()
        ax.set_ylim(0, 10)
        plt.show()

helper = F1_graphs(2022, 'Hungary', 'R')  
#helper.print_tyre_degredation()
#ref_laps = helper.plot_cummulative_diff(['VER', 'LEC', 'HAM', 'RUS', 'SAI', 'PER'], ref_time="1:24")

In [ ]:
helper = F1_graphs(2022, 'Hungary', 'R')

In [ ]:
drivers=['HAM', 'VER', 'LEC']
helper.plot_lap_times(drivers, filter=False)
helper.plot_lap_times(drivers, filter=True, ymin="1:20", ymax="1:30")

In [ ]:
dl = [{'driver': 'VER', 'lap': 'fastest'}, 
      {'driver': 'LEC', 'lap': 'fastest'}]
#helper.compare_laps(dl)
helper.compare_laps(dl, 2200, 2500)

In [ ]:
#helper.plot_circuit(2200, 4000, 7500, 10000)
helper.plot_circuit()

In [ ]:
helper.plot_cummulative_diff(['LEC', 'HAM', 'RUS', 'SAI', 'PER'])

In [ ]:
helper.plot_circuit(-500, 500, -500, 1500)

In [ ]:
dl = [{'driver': 'VER', 'lap': 41}, 
      {'driver': 'VER', 'lap': 42}]
#helper.compare_laps(dl)
helper.compare_laps(dl, 3700, 4100, 'Spin of Verstappen (lap 41)')

In [ ]:
dl = [{'driver': 'LEC', 'lap': 57}, 
      {'driver': 'LEC', 'lap': 43}]
helper.compare_laps(dl, 3700, 4100, title='Leclerc Red (lap 57) vs White (lap 43) tyres')

In [ ]:
helper.get_fastest_laps()

In [ ]:
helper.plot_cummulative_diff(['LEC', 'HAM', 'RUS', 'SAI', 'PER'], ref_time="1:24")

In [ ]:
helper.get_fastest_laps_per_stint()

In [ ]:
helper.get_best_possible_time_per_compound()

In [ ]:
helper.get_best_possible_time_per_driver()

In [ ]:
helper.plot_speed_on_map('LEC')

In [ ]:
helper.plot_gear_on_map('LEC')

In [ ]:
helper.plot_teldata_on_map('LEC', 'Throttle')

In [ ]:
helper.plot_brake_on_map('LEC')

In [ ]:
helper.plot_fastest_laps_relative()

In [ ]:
helper.print_tyre_degredation()

In [ ]:
helper.print_tyre_strategies()

In [ ]:
helper.print_pitstop_time_in_pits()

In [ ]:
helper.print_pitstop_time_per_team()

In [ ]:
helper.plot_top_speed()

In [ ]:
helper.plot_pitstop_timelost(['VER', 'HAM', 'RUS', 'SAI', 'LEC', 'PER'], 2)

In [ ]:
pitstops = helper.get_pitstop_info()
pitstops[pitstops.Driver.isin(['VER', 'HAM', 'LEC', 'RUS', 'SAI', 'PER'])].sort_values('OutLap') \
    [['Driver', 'StopCount', 'LapNumber', 'InLap', 'OutLap']]

In [ ]:
dl = [{'driver': 'VER', 'lap': 39}, 
      {'driver': 'HAM', 'lap': 20}]
helper.compare_laps_auto_color(dl)

In [ ]:
helper.get_pitstop_info()[['Driver', 'TimeInPit']].groupby('Driver').mean().sort_values('TimeInPit').head(5)

In [ ]:
helper2 = F1_graphs(2022, 'Hungary', 'FP2')
helper2.print_tyre_degredation(False)